In [1]:
import datetime as dt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.feature
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import xarray as xr
import math
import netCDF4
#from get_ellipse_metrics import get_emetrics_max_min
from fitEllipse2_new import fitEllipseContour
from geopy.distance import great_circle
from matplotlib.patches import Polygon
from matplotlib import cm #colormaps!
import copy
import os
import scipy.stats

#os.environ['DISPLAY']=':0.0'


In [2]:
#### Define functions first:
def point_inside_polygon(x,y,xarr,yarr):

	n = len(xarr)
	inside =False

	p1x = xarr[0]
	p1y = yarr[0]
	for i in range(n+1):
		p2x,p2y = xarr[i % n],yarr[i % n]
		if y > min(p1y,p2y):
			if y <= max(p1y,p2y):
				if x <= max(p1x,p2x):
					if p1y != p2y:
						xinters = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
					if p1x == p2x or x <= xinters:
						inside = not inside
		p1x,p1y = p2x,p2y

	return inside;


In [3]:
tempvals = np.zeros((20,484,64,134))

In [4]:
dec_ephi_ratio10 = np.zeros((20,124))
dec_ephi_wind10 = np.zeros((20,124))
dec_ephi_cenlat10 = np.zeros((20,124))
dec_ephi_size10 = np.zeros((20,124))
dec_ephi10 = np.zeros((20,124))

dec_ephi_ratio30 = np.zeros((20,124))
dec_ephi_wind30 = np.zeros((20,124))
dec_ephi_cenlat30 = np.zeros((20,124))
dec_ephi_size30 = np.zeros((20,124))
dec_ephi30 = np.zeros((20,124))

dec_ephi_ratio50 = np.zeros((20,124))
dec_ephi_wind50 = np.zeros((20,124))
dec_ephi_cenlat50 = np.zeros((20,124))
dec_ephi_size50 = np.zeros((20,124))
dec_ephi50 = np.zeros((20,124))

jan_ephi_ratio10 = np.zeros((20,124))
jan_ephi_wind10 = np.zeros((20,124))
jan_ephi_cenlat10 = np.zeros((20,124))
jan_ephi_size10 = np.zeros((20,124))
jan_ephi10 = np.zeros((20,124))

jan_ephi_ratio30 = np.zeros((20,124))
jan_ephi_wind30 = np.zeros((20,124))
jan_ephi_cenlat30 = np.zeros((20,124))
jan_ephi_size30 = np.zeros((20,124))
jan_ephi30 = np.zeros((20,124))

jan_ephi_ratio50 = np.zeros((20,124))
jan_ephi_wind50 = np.zeros((20,124))
jan_ephi_cenlat50 = np.zeros((20,124))
jan_ephi_size50 = np.zeros((20,124))
jan_ephi50 = np.zeros((20,124))

feb_ephi_ratio10 = np.zeros((20,112))
feb_ephi_wind10 = np.zeros((20,112))
feb_ephi_cenlat10 = np.zeros((20,112))
feb_ephi_size10 = np.zeros((20,112))
feb_ephi10 = np.zeros((20,112))

feb_ephi_ratio30 = np.zeros((20,112))
feb_ephi_wind30 = np.zeros((20,112))
feb_ephi_cenlat30 = np.zeros((20,112))
feb_ephi_size30 = np.zeros((20,112))
feb_ephi30 = np.zeros((20,112))

feb_ephi_ratio50 = np.zeros((20,112))
feb_ephi_wind50 = np.zeros((20,112))
feb_ephi_cenlat50 = np.zeros((20,112))
feb_ephi_size50 = np.zeros((20,112))
feb_ephi50 = np.zeros((20,112))

mar_ephi_ratio10 = np.zeros((20,124))
mar_ephi_wind10 = np.zeros((20,124))
mar_ephi_cenlat10 = np.zeros((20,124))
mar_ephi_size10 = np.zeros((20,124))
mar_ephi10 = np.zeros((20,124))

mar_ephi_ratio30 = np.zeros((20,124))
mar_ephi_wind30 = np.zeros((20,124))
mar_ephi_cenlat30 = np.zeros((20,124))
mar_ephi_size30 = np.zeros((20,124))
mar_ephi30 = np.zeros((20,124))

mar_ephi_ratio50 = np.zeros((20,124))
mar_ephi_wind50 = np.zeros((20,124))
mar_ephi_cenlat50 = np.zeros((20,124))
mar_ephi_size50 = np.zeros((20,124))
mar_ephi50 = np.zeros((20,124))

In [5]:
### The main code begins here....
home_dir = '/home11/grad/2020/ef935217/Research/Ellipses/'  # <----Change this 
lev_list = [10,50,30]
temp_lev = [850]
#contour_list = [31500,20000,23000]
contour_list = [30000,20000,23000]

write_out = True #write the ellipse data to a text file, True or False 

In [6]:
### Read in the data
## This must have a time component, or else fhr_inc<24h won't work.

year = 1999
month = 12
day = 1
date1 = dt.datetime(year,month,day,0)  #first date to plot
total_days = 31
first_fhr = 0
hours = total_days * 24
hr_inc = 6
times = [date1 + dt.timedelta(hours=x) for x in range(0,hours,hr_inc)]
date_list = netCDF4.date2num(times,units="hours since 1800-01-01 00:00:00",calendar="gregorian") #change dates to netcdf times


windlat = 60.0

#For real time plots...
#today = dt.datetime.utcnow()-dt.timedelta(days=0)   ### changed this to 0 day delay
#today = today.replace(hour=0,minute=0,second=0,microsecond=0)
#day_str = today.strftime("%Y%m%d")
#day_label = today.strftime("Init: %a %b %d %Y %H%M UTC")
#first_fhr = 0
#last_fhr = 384
#fhr_inc = 6
#fhr_list = np.arange(first_fhr,last_fhr+fhr_inc,fhr_inc)

  
metric_list = ["cenlat","cenlon","ratio","phi","area","u","t"]
min_data = {}
max_data = {}
avg_data = {}


In [7]:
##### temperature avg matrices
tmeanfile = xr.open_dataset("/erai/climo/mean/t.mean.climo.nc")
print(tmeanfile)
tmeandata = np.array(tmeanfile.isel(lev=6, lat = 234).t) 
tmean = np.mean(tmeandata)
print(tmean)
##produces an annual

print("850 temp data")
t2file = xr.open_dataset("/erai/"+str(year)+"/t."+str(year)+".nc")
t2_files = t2file["t"]
start = np.datetime64('1999-12-01T00:00:00')
stop = np.datetime64('1999-12-31T18:00:00')
t2_data = t2_files.loc[dict(time=slice(start,stop),lat=slice(19.5,64.86),lon=slice(198.24,292),lev=temp_lev)].squeeze(dim="lev")
t2lats = t2_data['lat'].values
print(t2lats)
tlats_era = t2_data["lat"].values
tlons_era = t2_data["lon"].values


tempvals[0,0:124,:,:] = t2_data[:,:,:]
#print(tempvals)

<xarray.Dataset>
Dimensions:  (hour: 4, lat: 256, lev: 37, lon: 512, time: 366)
Coordinates:
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 363.0 364.0 365.0 366.0
  * hour     (hour) float64 0.0 6.0 12.0 18.0
  * lev      (lev) int32 1000 975 950 925 900 875 850 825 ... 30 20 10 7 5 3 2 1
  * lat      (lat) float32 -89.46282 -88.76695 -88.06697 ... 88.76695 89.46282
  * lon      (lon) float32 0.0 0.70312524 1.4062505 ... 358.59387 359.297
Data variables:
    t        (time, hour, lev, lat, lon) float32 ...
261.84604
850 temp data
[19.99996  20.701714 21.403465 22.105219 22.806973 23.508724 24.210478
 24.91223  25.613983 26.315735 27.017488 27.71924  28.420994 29.122746
 29.8245   30.52625  31.228004 31.929756 32.631508 33.333263 34.035015
 34.736767 35.43852  36.14027  36.842022 37.543777 38.24553  38.94728
 39.649033 40.350784 41.052536 41.754288 42.45604  43.15779  43.859543
 44.561295 45.263046 45.964798 46.66655  47.3683   48.070053 48.771805
 49.473557 50.17531  50.87706  51

In [11]:
for plot_lev,the_contour in zip(lev_list,contour_list): #loop through each level
	print("Opening ERAI data "+str(plot_lev)+" and controur level "+str(the_contour)  )
	
	plots_loc10 = "./POR/10hPa/"   ##<-- Where are you saving the plot files?
	plots_loc30 = "./POR/30hPa/"
	plots_loc50 = "./POR/50hPa/"
	text_file = "./"+date1.strftime("%Y%m%d_%H%M")+"_"+str(plot_lev)+".txt"   ##<-- Where are you saving text files?
	fout = open(text_file,'w')
    ##temp text
	text_file2 = "./"+date1.strftime("%Y%m%d_%H%M")+"_temp"+str(plot_lev)+".txt"  
	fout2 = open(text_file2,'w')
	if plot_lev == 10:
 
		gfile = xr.open_dataset("/erai/"+str(year)+"/g."+str(year)+".nc")
		g_files = gfile["g"]
		tfile = xr.open_dataset("/erai/"+str(year)+"/t."+str(year)+".nc")
		t_files = tfile["t"]
		ufile = xr.open_dataset("/erai/"+str(year)+"/u."+str(year)+".nc")
		u_files = ufile["u"]


		print("Partitioning and averaging data.")
		print("First, g data")
		g_data = g_files.loc[dict(lat=slice(0,90),lev=lev_list)]
		print("u data")
		u_data = u_files.loc[dict(lat=slice(windlat-1,windlat+1),lev=lev_list)].mean(dim='lon')
		print("t data")        
		t_data = t_files.loc[dict(lat=slice(75,90),lev=lev_list)]
		tlats = t_data['lat'].values
		weights=np.cos(np.deg2rad(t_data.lat))
		weights.name = "weights"

        
#hgt_250, lon = cutil.add_cyclic_point(data_hght.variables['Geopotential_height_isobaric'][:],coord=lon)
		print("lat and lon")
		lats_era = g_data["lat"].values
		lons_era = g_data["lon"].values

        
	print(f"level: {plot_lev}")

	for date in date_list: #loop through each day 
		print(f"date: {date_list}")
		ind = np.where(date_list == date)[0]
		for d in ind:
			t = d
			height = g_data.loc[dict(time=times[t],lev=plot_lev)]
			u = u_data.loc[dict(time=times[t],lev=plot_lev)].mean(dim='lat')
			tdata = t_data.loc[dict(time=times[t],lev=plot_lev)].mean(dim='lon')   
			tdata2 = t2_data.loc[dict(time=times[t])]
		temp = tdata.weighted(weights).mean(dim='lat')
        
		print("t2data")
		print(tdata2)
		#stop
		td2 = tdata2.values
		for i in range(len(td2[:,1])):
			fout2.write(str(td2[i,:])+'\n')            

              
		formatted_date = times[t].strftime("%Y%m%d")
		emark = []
		eline = []
		cs_temp = []
		#mem_list = list[range(0,30),31]
		#mem_list.append(0)
		#mem_list.append(-1)
		print("##########################")
		print(date)
		print("Date "+times[t].strftime("%H UTC %d %b %Y"))
		valid_label = (times[t].strftime("Valid: %H UTC %d %b %Y"))
		
		
		plt.Figure(figsize=(15,15),dpi=120)#figure(figsize=(12,12),dpi=1200) <---Set fig size here!

		#ax = plt.subplot(1,1,1,projection=ccrs.Orthographic(0,90))
		ax = plt.axes(projection=ccrs.Orthographic(0,90))
		ax.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='dimgray',facecolor='none')
		#ax.set_extent([-180,180,0,90],ccrs.PlateCarree())
		ax.outline_patch.set_edgecolor('none')
		gl = ax.gridlines(color="grey",linestyle=":",linewidth=0.5)#=lat_lon_width)

		clevs = range(18000,33500,250)

		[x,y] = np.meshgrid(lons_era,lats_era)
		[x2,y2] = np.meshgrid(tlons_era,tlats_era)


		plt.title(str(plot_lev)+"hPa Elliptical Diagnostics\n"+valid_label)

		mem_color = "blue"
		mem_lw = 3.0
		mem_ms = 8
		cont_color = "#aec2e2"
		cont_lw = 1.0
        
		cs_temp.append(ax.contour(x,y,height,levels=clevs,linewidths=cont_lw,colors=cont_color))
		

		line_w = np.array(clevs,dtype=np.float)
		for i,c in enumerate(clevs):
			if c == the_contour:
				line_w[i] = 1.5 # Make the contour we're using for the ellipse calc fatter
			else:
				line_w[i] = 0.5
		print(clevs)

		ax.contour(x,y,height,clevs,transform=ccrs.PlateCarree(),extend='both',colors='black',linewidths=line_w)
		ax.contour(x2,y2,tdata2, lev = [tmean], transform=ccrs.PlateCarree(),extend='both',colors='blue',linewidths=line_w)
##NEED TO INCORPORATE THE CLIMATOLOGICAL MEAN TEMP LINE AT 850 MB
		try:
			lev_contour_ind = np.where(np.array(cs_temp[-1].levels)==the_contour)[0][0]
			isoline_list = cs_temp[-1].allsegs[lev_contour_ind]
			print("in try")
			print("Number of ellipses: ",len(isoline_list))                                
		except:
			print("Contours not found for",the_contour,"meter level at",plot_lev,"hPa.")
			isoline_list = []        
        
		if len(isoline_list) > 0:
			lev_contour_ind = np.where(np.array(cs_temp[-1].levels)==the_contour)[0][0]
			isoline_list = cs_temp[-1].allsegs[lev_contour_ind]
			#print(isoline_list)
			print("in elipse ...")
			#quit()
			print("Number of ellipses: ",len(isoline_list))                                
		else:
			print("Contours not found for",the_contour,"meter level at",plot_lev,"hPa.")
			isoline_list = []

		isocount = 0
		small = 0
		for isoline in isoline_list:
			#[iso_lon,iso_lat] = mm(isoline[:,0],isoline[:,1],inverse=True)
			[iso_lon,iso_lat] = [isoline[:,0],isoline[:,1]]
			if len(iso_lon)<15:
				print("-----Not analyzing ellipse with",len(iso_lon),"points, continuing...")  # Check for size!
				small = 1
				continue

			# Checking to see if contours are closed (0) or need to be joined (1)  Checking before convert lat/lon to radians
			lon_diff = abs(iso_lon[0] - iso_lon[len(iso_lon)-1])
			lat_diff = abs(iso_lat[0] - iso_lat[len(iso_lat)-1])
			join = 0
			if lon_diff > 1 or lat_diff > 1:
				join = 1
				print("Diffs lat/lon: ",lat_diff,lon_diff)
			iso_lon = np.deg2rad(iso_lon)
			iso_lat = np.deg2rad(iso_lat)

	
			## Now convert it into polar coordinates for ellipse math.
			ex = np.array((np.cos(iso_lon)*np.cos(iso_lat))/(1+np.sin(iso_lat)))
			ey = np.array((np.sin(iso_lon)*np.cos(iso_lat))/(1+np.sin(iso_lat)))

			## does this contour include the pole?
			overpole = point_inside_polygon(0,0,ex,ey)  #returns true if poly includes the pole, false if not 
			print("It is",overpole, "that the contour includes the pole")
			   
			### contours break if they don't include the pole and need to be reconnected here...
			### first check that contour doesn't include pole, there is more than one contour,... 
			###        ...the contours are not a real split vortex and we didn't get rid of one because it was small...
			if not overpole and len(isoline_list) > 1 and join > 0 and small < 1:
				if isocount > 0:
					ex = np.append(ex,ex2)
					ey = np.append(ey,ey2)
					print("Point didn't include pole - add to these to previous set")
				else:
					ex2 = ex
					ey2 = ey
					print("Point didn't include pole - keeping these to add to next set")
					isocount = 1
					continue

			print("Running ellipse diagnostic now")
			exx,eyy,eaax,ebax,ecenterx,ecentery,ephi = fitEllipseContour(ex,ey)
			
			## Convert back to lat/lon 
			elons = np.where(exx<0,np.where(eyy>0,np.arctan(eyy/exx)+math.pi,np.arctan(eyy/exx)-math.pi),np.arctan(eyy/exx))
			yysinxxlon = eyy/np.sin(elons)
			elats = -2*(np.arctan(yysinxxlon) - (math.pi/4.0))
			elats = np.rad2deg(elats)
			elons = np.rad2deg(elons)
	
			## Still not really sure what this is for...
			for g in range(1,len(elats)):
				if abs(elats[g]-elats[g-1]) > 1.5:
					elats[g] = elats[g-1]
	
			## Center points back to lat/lon
			cenlon = np.where(ecenterx<0,np.where(ecentery>0,np.arctan(ecentery/ecenterx)+math.pi,np.arctan(ecentery/ecenterx)-math.pi),np.arctan(ecentery/ecenterx))
			ysinlon = ecentery/np.sin(cenlon)
			cenlat = np.rad2deg(-2 * (np.arctan(ysinlon) - (math.pi/4.0)))
			cenlon = np.rad2deg(cenlon)
			print("Center of ellipse:",cenlat,"N",cenlon,"E")
	
			## Calculate endpoints of the axes of the vortex, convert to lat/lon
			xa = eaax * np.cos(ephi)
			ya = eaax * np.sin(ephi)
			xb = ebax * np.sin(ephi)
			yb = ebax * np.cos(ephi)
			endx = np.array([ecenterx+xa,ecenterx-xa,ecenterx+xb,ecenterx-xb])
			endy = np.array([ecentery+ya,ecentery-ya,ecentery-yb,ecentery+yb])
			endlon = np.where(endx < 0,np.where(endy>0,np.arctan(endy/endx)+math.pi,np.arctan(endy/endx)-math.pi),np.arctan(endy/endx))
			ysinelon = endy/np.sin(endlon)
			endlat = np.rad2deg(-2 *(np.arctan(ysinelon) - (math.pi/4)))
			endlon = np.rad2deg(endlon)
	
			## Calc great circle distances; (still necessary?) 
			a1gc = great_circle((endlat[0],endlon[0]),(cenlat,cenlon)).km
			a2gc = great_circle((cenlat,cenlon),(cenlat,endlat[1])).km
			b1gc = great_circle((endlat[2],endlon[2]),(cenlat,cenlon)).km
			b2gc = great_circle((cenlat,cenlon),(endlat[3],endlon[3])).km

			ephi = np.rad2deg(ephi)
			## If a1 shorter than b1, adjust phi to correct the orientation angle
			if a1gc < b1gc:
				ephi -= 90
			print("Emetrics phi:",ephi)
			if ephi < -45:
				ephi += 180
			
			ratio = a1gc/b1gc
			if ratio < 1.0:
				ratio = 1.0/ratio
	
			size = math.pi*a1gc*b1gc
			estr = times[t].strftime("%H %Y%m%d%H")+" "+f'{cenlon:.3f}'+" "+f'{cenlat:.3f}'+" "+f'{a1gc:.3f}'+" "+f'{b1gc:.3f}'+" "+f'{ephi:.3f}'+" "+f'{u.values:.3f}'+" "+f'{temp.values:.3f}'
			if write_out:
				fout.write(estr+"\n")

			exy=np.array(list(zip(elons,elats)))
			plt.plot(elons,elats,color='red',transform=ccrs.Geodetic(),linewidth=mem_lw)
			plt.plot(cenlon,cenlat,markersize=4,marker='o',color='red',transform=ccrs.PlateCarree())
			#ax.add_patch(Polygon(exy,closed=True,color='magenta',fill=False,lw=mem_lw,transform=ccrs.PlateCarree()))
		
		if plot_lev == 10:
			dec_ephi_ratio10[0,:] = ratio
			dec_ephi_wind10[0,:] = u.values
			dec_ephi_cenlat10[0,:] = cenlat
			dec_ephi_size10[0,:] = size
			dec_ephi10[0,:] = ephi

		if plot_lev == 30:
			dec_ephi_ratio30[0,:] = ratio
			dec_ephi_wind30[0,:] = u.values
			dec_ephi_cenlat30[0,:] = cenlat
			dec_ephi_size30[0,:] = size
			dec_ephi30[0,:] = ephi

            
		if plot_lev == 50:
			dec_ephi_ratio50[0,:] = ratio
			dec_ephi_wind50[0,:] = u.values
			dec_ephi_cenlat50[0,:] = cenlat
			dec_ephi_size50[0,:] = size
			dec_ephi50[0,:] = ephi

			
		#print(cenlon)
		#print(cenlat)
		#print(ephi)
		print("right before savefig")
		plot_label = times[t].strftime("%Y%m%d%H")
        
		if plot_lev == 10:
			plt.savefig(plots_loc10+f"{plot_label}_{plot_lev}hpa.png",format='png')            
		if plot_lev == 30:
			plt.savefig(plots_loc30+f"{plot_label}_{plot_lev}hpa.png",format='png')
		if plot_lev == 50:
			plt.savefig(plots_loc50+f"{plot_label}_{plot_lev}hpa.png",format='png')
            
		#plt.savefig(plots_loc+"ellipse"+str(plot_lev)+"_"+str(fhr/6)+".png",format='png')
		print("right after savefig")
		#plt.close()
		plt.clf()
		#quit()
	#quit()

Opening ERAI data 10 and controur level 30000
Partitioning and averaging data.
First, g data
u data
t data
lat and lon
level: 10
date: [1752408. 1752414. 1752420. 1752426. 1752432. 1752438. 1752444. 1752450.
 1752456. 1752462. 1752468. 1752474. 1752480. 1752486. 1752492. 1752498.
 1752504. 1752510. 1752516. 1752522. 1752528. 1752534. 1752540. 1752546.
 1752552. 1752558. 1752564. 1752570. 1752576. 1752582. 1752588. 1752594.
 1752600. 1752606. 1752612. 1752618. 1752624. 1752630. 1752636. 1752642.
 1752648. 1752654. 1752660. 1752666. 1752672. 1752678. 1752684. 1752690.
 1752696. 1752702. 1752708. 1752714. 1752720. 1752726. 1752732. 1752738.
 1752744. 1752750. 1752756. 1752762. 1752768. 1752774. 1752780. 1752786.
 1752792. 1752798. 1752804. 1752810. 1752816. 1752822. 1752828. 1752834.
 1752840. 1752846. 1752852. 1752858. 1752864. 1752870. 1752876. 1752882.
 1752888. 1752894. 1752900. 1752906. 1752912. 1752918. 1752924. 1752930.
 1752936. 1752942. 1752948. 1752954. 1752960. 1752966. 1752972

/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:77: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:80: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.
/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1478: UserWarning: The following kwargs were not used by contour: 'lev'
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


in try
Number of ellipses:  1
in elipse ...
Number of ellipses:  1
Diffs lat/lon:  0.05066574482015085 359.2969970703125
It is True that the contour includes the pole
Running ellipse diagnostic now
old center = [0.13397896 0.0878983 ]
old angle of rotation = -35.0198813918669
old axes = [0.34963653 0.25047763]
center = [0.13397896 0.0878983 ]
angle of rotation = -35.0198813918669
axes = [0.34963653 0.25047763]
0.3496365253391567 0.2504776328112609 0.13397896054351982 0.08789829748233438 -0.611212233945972
Center of ellipse: 71.79276589210845 N 33.26729625510287 E
Emetrics phi: -35.0198813918669
right before savefig
right after savefig
date: [1752408. 1752414. 1752420. 1752426. 1752432. 1752438. 1752444. 1752450.
 1752456. 1752462. 1752468. 1752474. 1752480. 1752486. 1752492. 1752498.
 1752504. 1752510. 1752516. 1752522. 1752528. 1752534. 1752540. 1752546.
 1752552. 1752558. 1752564. 1752570. 1752576. 1752582. 1752588. 1752594.
 1752600. 1752606. 1752612. 1752618. 1752624. 1752630. 1752

/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:80: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.
/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1478: UserWarning: The following kwargs were not used by contour: 'lev'
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


range(18000, 33500, 250)
in try
Number of ellipses:  1
in elipse ...
Number of ellipses:  1
Diffs lat/lon:  0.02257621950830213 359.2969970703125
It is True that the contour includes the pole
Running ellipse diagnostic now
old center = [0.13693007 0.08479109]
old angle of rotation = -32.84623796074114
old axes = [0.3545819  0.25128921]
center = [0.13693007 0.08479109]
angle of rotation = -32.84623796074114
axes = [0.3545819  0.25128921]
0.35458189898906883 0.25128920619467787 0.13693007082578773 0.08479109430491667 -0.5732749993084808
Center of ellipse: 71.70135995351407 N 31.76699974324221 E
Emetrics phi: -32.846237960741135
right before savefig
right after savefig
date: [1752408. 1752414. 1752420. 1752426. 1752432. 1752438. 1752444. 1752450.
 1752456. 1752462. 1752468. 1752474. 1752480. 1752486. 1752492. 1752498.
 1752504. 1752510. 1752516. 1752522. 1752528. 1752534. 1752540. 1752546.
 1752552. 1752558. 1752564. 1752570. 1752576. 1752582. 1752588. 1752594.
 1752600. 1752606. 1752612.

...

right after savefig
date: [1752408. 1752414. 1752420. 1752426. 1752432. 1752438. 1752444. 1752450.
 1752456. 1752462. 1752468. 1752474. 1752480. 1752486. 1752492. 1752498.
 1752504. 1752510. 1752516. 1752522. 1752528. 1752534. 1752540. 1752546.
 1752552. 1752558. 1752564. 1752570. 1752576. 1752582. 1752588. 1752594.
 1752600. 1752606. 1752612. 1752618. 1752624. 1752630. 1752636. 1752642.
 1752648. 1752654. 1752660. 1752666. 1752672. 1752678. 1752684. 1752690.
 1752696. 1752702. 1752708. 1752714. 1752720. 1752726. 1752732. 1752738.
 1752744. 1752750. 1752756. 1752762. 1752768. 1752774. 1752780. 1752786.
 1752792. 1752798. 1752804. 1752810. 1752816. 1752822. 1752828. 1752834.
 1752840. 1752846. 1752852. 1752858. 1752864. 1752870. 1752876. 1752882.
 1752888. 1752894. 1752900. 1752906. 1752912. 1752918. 1752924. 1752930.
 1752936. 1752942. 1752948. 1752954. 1752960. 1752966. 1752972. 1752978.
 1752984. 1752990. 1752996. 1753002. 1753008. 1753014. 1753020. 1753026.
 1753032. 1753038. 175304

/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:80: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.


range(18000, 33500, 250)
in try
Number of ellipses:  1
in elipse ...
Number of ellipses:  1
Diffs lat/lon:  0.06761930141276196 359.2969970703125
It is True that the contour includes the pole
Running ellipse diagnostic now
old center = [0.055104   0.02039389]
old angle of rotation = 1.7668115450755404
old axes = [0.2234997  0.36073338]
center = [0.055104   0.02039389]
angle of rotation = -88.23318845492446
axes = [0.36073338 0.2234997 ]
0.3607333790607532 0.22349969570850314 0.055104004044418196 0.020393894907036768 -1.5399596480710802
Center of ellipse: 83.27469831638341 N 20.309424497528752 E
Emetrics phi: -88.23318845492446
right before savefig


/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1478: UserWarning: The following kwargs were not used by contour: 'lev'
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


right after savefig


In [ ]:
ephi_multi = []
ephi_multi.extend(dec_ephi50)
ephi_multi.extend(jan_ephi50)
ephi_multi.extend(feb_ephi50)
ephi_multi.extend(mar_ephi50)

In [ ]:
#trying the regression
corr = np.zeros((10,21))
pval = np.empty((10,21))
pval[:] = np.nan

for i in range(0,10):
    for j in range(0,21):
        reg_temps = tempvals[:,i,j]
        rt = []
        ep = []
        for k in range(0,484):
            if k <= 20:
                continue
            else:
                t = k-20
                rt.append(reg_temps[k])
                ep.append(ephi_multi[t])
        #print(len(rt))
        #print(len(ep))
        test = scipy.stats.pearsonr(rt, ep)
        #print(test[0])
        corr[i,j] = test[0]
        if test[1] <= 0.05:
            pval[i,j] = test[1]

lats = t2_data['lat'].values
lons = t2_data['lon'].values

print(pval)

In [ ]:
##plot contour of regression
fig2, ax2 = plt.subplots()
a = ax2.contourf(lons, lats, corr)

cbar = fig2.colorbar(a)
cbar.ax.set_ylabel('Correlation Coefficient')

# use contourf() with proper hatch pattern and alpha value
cs = ax2.contourf(lons, lats, pval, hatches=['..'], colors = 'none')

plt.xlabel("Longitudes $^o$W")
plt.ylabel("Latitudes $^o$N")
plt.title("Pearsons Correlation")

plt.savefig("50hPa_Full2006_PreviousEphi.png")